In [2]:
!pip install transformers[torch] accelerate datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

In [3]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [16]:
from datasets import load_dataset

#dataset = load_dataset('wmt18', 'zh-en', split='train[:1%]')
#print(f"数据集大小: {len(dataset)}")

# 打印前几个示例以确认数据加载成功
print(f"数据集示例: {dataset[:10]}")


数据集示例: {'translation': [{'en': '1929 or 1989?', 'zh': '1929年还是1989年?'}, {'en': 'PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.', 'zh': '巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。'}, {'en': 'At the start of the crisis, many people likened it to 1982 or 1973, which was reassuring, because both dates refer to classical cyclical downturns.', 'zh': '一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。'}, {'en': 'Today, the mood is much grimmer, with references to 1929 and 1931 beginning to abound, even if some governments continue to behave as if the crisis was more classical than exceptional.', 'zh': '如今人们的心情却是沉重多了，许多人开始把这次危机与1929年和1931年相比，即使一些国家政府的表现仍然似乎把视目前的情况为是典型的而看见的衰退。'}, {'en': 'The tendency is either excessive restraint (Europe) or a diffusion of the effort (the United States).', 'zh': '目前的趋势是，要么是过度的克制（欧洲），要么是努力的扩展（美国）。'}, {'en': 'Europe is being cautious in

max_length 这边不宜设定太高这样后面训练的时候容易内存爆掉，参数用于指定切分后的文本序列的最大长度。如果输入文本的长度超过了max_length，则会进行截断（truncation）以确保序列的长度不超过max_length。如果输入文本的长度不足max_length，将会进行填充（padding）以使序列长度一致。

In [5]:
# 加载预训练的T5分词器
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# 预处理函数，添加任务前缀
def preprocess_function(examples):
    inputs = ["translate Chinese to English: " + ex['zh'] for ex in examples['translation']]
    targets = [ex['en'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs
# 处理数据集
encoded_dataset = dataset.map(preprocess_function, batched=True)

# 打印预处理后的前几个示例
print(f"预处理后的数据集示例: {encoded_dataset[:3]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/251603 [00:00<?, ? examples/s]

预处理后的数据集示例: {'translation': [{'en': '1929 or 1989?', 'zh': '1929年还是1989年?'}, {'en': 'PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.', 'zh': '巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。'}, {'en': 'At the start of the crisis, many people likened it to 1982 or 1973, which was reassuring, because both dates refer to classical cyclical downturns.', 'zh': '一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。'}], 'input_ids': [[13959, 2830, 12, 1566, 10, 27332, 2, 2294, 3914, 2, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13959, 2830, 12, 1566, 10, 3, 2, 18, 2, 

In [6]:
# 加载预训练的T5模型
model = T5ForConditionalGeneration.from_pretrained('t5-small', force_download=True).to(device)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,  # 增加批处理大小
    per_device_eval_batch_size=128,   # 增加批处理大小
    num_train_epochs=10,  # 设置为10个epoch进行训练
    weight_decay=0.01,
    fp16=True,  # 启用混合精度训练
    gradient_accumulation_steps=2,  # 累积2步梯度
)

# 创建Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
)

# 训练模型
print("开始训练模型...")
trainer.train()
print("训练完成。")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

开始训练模型...


Epoch,Training Loss,Validation Loss
1,1.519000,0.898412
2,0.953000,0.883141
3,0.937900,0.875646
4,0.930500,0.870910
5,0.927400,0.867543
6,0.922500,0.865184
7,0.920600,0.863450
8,0.918800,0.862301
9,0.917300,0.861620
10,0.916300,0.861392


训练完成。


In [17]:
# 翻译函数
def translate_texts(texts):
    # 确保模型在正确的设备上
    model.to(device)

    # 添加任务前缀
    texts = ["translate Chinese to English: " + text for text in texts]

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    # 删除不必要的参数
    inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}

    # 检查输入张量
    print(f"输入张量: {inputs}")

    # 调整生成参数
    outputs = model.generate(**inputs, max_length=128, num_beams=10, early_stopping=True)

    # 检查生成的输出张量
    print(f"生成的输出张量: {outputs}")

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# 示例文本
examples = ["巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。",
            "一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。",
            "今天是个好天气","令人宽心的","1929年还是1989年?","巴黎",
            "然而，和1989年一样，2008-2009年很可能也能被视为一个划时代的改变，其带来的发人深省的后果将在几十年后仍能让我们感受得到。"]
translations = translate_texts(examples)

# 检查翻译结果
for i, (src, tgt) in enumerate(zip(examples, translations)):
    print(f"示例 {i+1}")
    print(f"中文: {src}")
    print(f"英文: {tgt}")
    print("-" * 20)

输入张量: {'input_ids': tensor([[13959,  2830,    12,  1566,    10,     3,     2,    18,     2,     6,
             2,     1,     0,     0,     0,     0,     0,     0,     0,     0],
        [13959,  2830,    12,  1566,    10,     3,     2,     6,     2,  2294,
          4613,     2,  2294,  4552,     2,     6,     2,     6,     2,     1],
        [13959,  2830,    12,  1566,    10,     3,     2,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [13959,  2830,    12,  1566,    10,     3,     2,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [13959,  2830,    12,  1566,    10, 27332,     2,  2294,  3914,     2,
            58,     1,     0,     0,     0,     0,     0,     0,     0,     0],
        [13959,  2830,    12,  1566,    10,     3,     2,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [13959,  2830,    

In [12]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
# 指定保存路径
import os
save_path = '/content/drive/MyDrive/Colab Notebooks/中翻英_finetune_t5_small_2'
os.makedirs(save_path, exist_ok=True)

# 保存模型和分词器
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"模型和分词器已保存到 {save_path}")

模型和分词器已保存到 /content/drive/MyDrive/Colab Notebooks/中翻英_finetune_t5_small_2
